# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [1]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [2]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list(set(training_data))

In [3]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

['3', '\n', 'h', 'b', ')']
44


In [4]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [5]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [6]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [7]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [8]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [9]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([ 1, 28, 11, 23, 43, 22, 20, 35, 25, 26])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [10]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [11]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[ 1, 28, 11, 23, 43, 22, 20, 35],
        [28, 11, 23, 43, 22, 20, 35, 25],
        [11, 23, 43, 22, 20, 35, 25, 26],
        [23, 43, 22, 20, 35, 25, 26, 43],
        [43, 22, 20, 35, 25, 26, 43, 31]])
y training data
tensor([[28, 11, 23, 43, 22, 20, 35, 25],
        [11, 23, 43, 22, 20, 35, 25, 26],
        [23, 43, 22, 20, 35, 25, 26, 43],
        [43, 22, 20, 35, 25, 26, 43, 31],
        [22, 20, 35, 25, 26, 43, 31, 30]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [12]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [13]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[ 1.3850,  0.0371, -0.3288,  0.0185, -0.3415, -0.4965, -0.1411,
          -0.9759],
         [ 1.2112, -0.4846, -0.4750, -0.9843, -1.2826, -1.8824,  0.0106,
          -0.3175],
         [ 0.5049,  0.5368, -2.1391,  1.0405, -1.4376,  0.2521,  0.6477,
          -1.8549],
         [-0.5855, -0.4191, -0.3631, -0.4719,  0.1352, -1.7974,  1.5508,
          -0.7135]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [14]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [15]:
x_embeddings = embeddings_table(x_batch)

In [16]:
# just one example
x_embeddings[:1]

tensor([[[ 1.2112, -0.4846, -0.4750, -0.9843, -1.2826, -1.8824,  0.0106,
          -0.3175],
         [ 0.4733, -0.1199, -0.5449,  0.8507,  2.0888,  0.8237, -2.1190,
           2.5619],
         [-1.9832,  0.2620,  0.1380,  0.9584, -0.5480, -0.4870, -1.1590,
          -1.1322],
         [ 1.7188, -0.3890, -2.4164,  0.5019,  0.1264,  0.4454, -0.4997,
          -0.3267],
         [-1.4688,  0.6228, -1.5106,  0.0222,  0.7943,  0.1544, -1.4003,
          -0.9542],
         [ 1.2867, -0.1125, -0.6056, -0.6780, -0.7280, -0.3151, -0.3150,
          -1.4323],
         [-1.1710,  0.4043, -0.3690, -0.8718, -0.1530,  0.5303,  0.5291,
           0.3108],
         [ 0.0107,  0.3701,  0.3706, -0.3482,  0.7253, -0.5952,  0.1215,
          -0.2093]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [17]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [18]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [19]:
d_k = d_q = d_v = 10

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, d_v, bias = False)


In [20]:
W_q.weight.shape

torch.Size([10, 8])

In [21]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [22]:
W_q.weight@x_embeddings_total[0][0]

tensor([ 1.1127, -1.0174, -1.1338, -0.6189,  2.2388,  0.3574,  1.3070, -0.9240,
         0.2629,  0.8002], grad_fn=<MvBackward0>)

In [23]:
W_q(x_embeddings_total)

tensor([[[ 1.1127, -1.0174, -1.1338, -0.6189,  2.2388,  0.3574,  1.3070,
          -0.9240,  0.2629,  0.8002],
         [-0.8734,  1.4688,  0.3898, -1.1546, -0.1878,  0.2352, -2.4749,
           3.0575, -0.7629, -0.8367],
         [-0.5283, -0.7939, -0.4795,  1.0361, -1.4450, -0.4216, -0.0783,
          -1.9665, -0.4330,  0.0668],
         [-0.4312, -0.9789,  0.2670, -1.1728, -0.0548,  2.0309, -0.4717,
          -0.4128,  0.5101, -0.1427],
         [ 1.8921, -0.3380, -0.0991, -0.4780,  1.3659,  0.8478, -0.3775,
           0.7474, -0.2129, -0.1943],
         [ 0.4803, -0.8005,  0.4770, -0.6908,  0.3881,  1.4207,  0.6942,
          -0.3384,  0.6829, -0.2487],
         [ 1.1085, -0.0606, -0.2274,  0.3974,  1.6677, -0.4606,  1.4423,
          -0.9333,  0.2122,  0.3233],
         [ 0.1732, -0.0884, -0.2788,  0.7088, -0.1170, -0.8600,  0.9556,
           0.0862,  0.1526,  0.7319]],

        [[-0.8316,  0.1231, -1.6983, -1.3082,  1.6614, -0.1181, -1.3862,
           0.3080, -1.0848,  0.1886],

In [24]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

In [25]:
print(Q.shape, K.shape, V.shape)

torch.Size([5, 8, 10]) torch.Size([5, 8, 10]) torch.Size([5, 8, 10])


In [26]:
attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

Let me try and explain what just happened above, let's take example of a single batch

In [29]:
print(Q[0][:3])
print(K[0][:3])

tensor([[ 1.1127, -1.0174, -1.1338, -0.6189,  2.2388,  0.3574,  1.3070, -0.9240,
          0.2629,  0.8002],
        [-0.8734,  1.4688,  0.3898, -1.1546, -0.1878,  0.2352, -2.4749,  3.0575,
         -0.7629, -0.8367],
        [-0.5283, -0.7939, -0.4795,  1.0361, -1.4450, -0.4216, -0.0783, -1.9665,
         -0.4330,  0.0668]], grad_fn=<SliceBackward0>)
tensor([[-0.1860, -0.4885,  0.1980,  0.0586,  0.4341, -0.5913,  1.2827, -0.8974,
         -0.2662, -0.2137],
        [-0.1443,  0.1097, -0.3931, -1.6552, -1.2581, -0.2229, -1.3502, -1.6376,
         -1.8208,  0.5214],
        [-0.1349,  1.1512, -0.5468,  0.2797,  0.2572,  1.4468, -1.4674,  0.6069,
          0.7681,  0.5911]], grad_fn=<SliceBackward0>)


well the traditional explanation is:

when the static embedding for a word is passed through these layers it extract specific feature pertaining to corresponding transformation

let's take an example: "The bank of the River"

Query -> transform the static embedding of bank to something like "I am a Noun needing a definition, need to know what am I a river bank, a financial bank, etc"

Key -> transforms the static embedding of River to something like "I am a nature related word, related to river"

Value -> transforms the embedding to actual meaning info, as their it might not be the first attention layer, if it is a second layer, it won't be the absolute meaning

but this never really sat with me completely, I was not able to understand this fully and it seemed pretty handwavy as to explaining what these layers really do, maybe I understand for Value vector but not for Query and Key vector

need to think and understand this properly and clearly once and for all, 
think, what would happen if no linear layer was present [next]

In [ ]:
# now each element of this of shape context length x context length
attention_matrix.shape

torch.Size([5, 8, 8])